# Template Tool

The `uwtools` API's `template` module provides functions to render Jinja2 templates and to translate atparse templates to Jinja2.

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.6.0. </div>
<div class="alert alert-info">For more information, please see the <a href="https://uwtools.readthedocs.io/en/2.5.0/sections/user_guide/api/template.html">uwtools.api.template</a> Read the Docs page.</div>

## Table of Contents

* [Rendering Values](#Rendering-Values)
* [Rendering to a String](#Rendering-to-a-String)
* [Translating atparse to Jinja2](#Translating-atparse-to-Jinja2)

In [1]:
import os
from pathlib import Path
from freezegun import freeze_time
from uwtools.api import template

if os.environ.get("FAKE_NOW"):
    freeze_time("2000-01-01 00:00:00 UTC").start()

## Rendering Values

`template.render()` renders a Jinja2 template using values provided by the specified values source.

In [2]:
help(template.render)

Help on function render in module uwtools.api.template:

render(values_src: 'dict | Path | str | None' = None, values_format: 'str | None' = None, input_file: 'Path | str | None' = None, output_file: 'Path | str | None' = None, cycle: 'datetime | None' = None, leadtime: 'timedelta | None' = None, overrides: 'dict[str, str] | None' = None, env: 'bool' = False, searchpath: 'list[str] | None' = None, values_needed: 'bool' = False, dry_run: 'bool' = False, stdin_ok: 'bool' = False) -> 'str'
    Render a Jinja2 template to a file, based on specified values.
    
    Primary values used to render the template are taken from the specified file. The format of the
    values source will be deduced from the filename extension, if possible. This can be overridden
    via the ``values_format`` argument. A ``dict`` object may alternatively be provided as the
    primary values source. If no input file is specified, ``stdin`` is read. If no output file is
    specified, ``stdout`` is written to.
   

Consider the following template, to be rendered as YAML data:

In [3]:
%%bash
cat fixtures/template/render-template.yaml

user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}


The `values_needed` parameter can be used to display which values are needed to complete the template. A logger needs to be initialized for the log of the missing values to be displayed.

In [4]:
import uwtools.logging
uwtools.logging.setup_logging(verbose=False)

print(
    template.render(
        input_file='fixtures/template/render-template.yaml',
        values_needed=True
    )
)

[2000-01-01T00:00:00]     INFO Value(s) needed to render this template are:
[2000-01-01T00:00:00]     INFO   first
[2000-01-01T00:00:00]     INFO   food
[2000-01-01T00:00:00]     INFO   last


user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}



The log messages indicate that values are needed for keys `first`, `food`, and `last`. These values can be sourced from a Python dictionary or from a file. The following file provides the needed values:

In [5]:
%%bash
cat fixtures/template/render-values.yaml

first: John
last: Doe
food: burritos


With these values, we can render the template to a file. When the source of values is a file, its path can be given either as a string or a <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. If it has an unrecognized (or no) extension, its format can be specified with `values_format`. The rendered template can be written to a file specified with `output_file`; otherwise, it will be written to `stdout`. 

In [6]:
print(
    template.render(
        values_src=Path('fixtures/template/render-values.yaml'),
        values_format='yaml',
        input_file='fixtures/template/render-template.yaml',
        output_file='fixtures/template/complete-render-1.yaml'
    )
)

user:
  name: John Doe
  favorite_food: burritos


Values can be selectively overridden with a dictionary passed via the optional `overrides` argument.

In [7]:
print(
    template.render(
        values_src=Path('fixtures/template/render-values.yaml'),
        values_format='yaml',
        input_file='fixtures/template/render-template.yaml',
        output_file='fixtures/template/complete-render-2.yaml',
        overrides={'first':'Jane', 'food':'tamales'}
    )
)

user:
  name: Jane Doe
  favorite_food: tamales


Let's take a look at the two newly rendered files.

In [8]:
%%bash
cat fixtures/template/complete-render-1.yaml
echo ---------------------------------------
cat fixtures/template/complete-render-2.yaml

user:
  name: John Doe
  favorite_food: burritos
---------------------------------------
user:
  name: Jane Doe
  favorite_food: tamales


## Rendering to a String

`template.render_to_str()` is identical to `template.render()` except that it does not accept an `output_file` parameter: It returns the rendered template as a string and does not write to a file or to `stdout`.

In [9]:
help(template.render_to_str)

Help on function render_to_str in module uwtools.api.template:

render_to_str(values_src: 'dict | Path | str | None' = None, values_format: 'str | None' = None, input_file: 'Path | str | None' = None, overrides: 'dict[str, str] | None' = None, env: 'bool' = False, searchpath: 'list[str] | None' = None, values_needed: 'bool' = False, dry_run: 'bool' = False) -> 'str'
    Render a Jinja2 template to a string, based on specified values.
    
    See ``render()`` for details on arguments, etc.



We can see the resulting string using the same template and values from the first `template.render()` example.

In [10]:
result = template.render_to_str(
    values_src=Path('fixtures/template/render-values.yaml'),
    values_format='yaml',
    input_file='fixtures/template/render-template.yaml'
)
print(result)

user:
  name: John Doe
  favorite_food: burritos


For more examples, please refer to the <a href='#Rendering-Values'>Rendering Values</a> section above.

## Translating atparse to Jinja2

The `translate()` function can be used to translate atparse templates into Jinja2 templates by replacing `@[]` tokens with their corresponding `{{}}` Jinja2 equivalents. 

In [11]:
help(template.translate)

Help on function translate in module uwtools.api.template:

translate(input_file: 'Path | str | None' = None, output_file: 'Path | str | None' = None, dry_run: 'bool' = False, stdin_ok: 'bool' = False) -> 'bool'
    Translate an atparse template to a Jinja2 template.
    
    ``@[]`` tokens are replaced with Jinja2 ``{{}}`` equivalents. If no input file is specified,
    ``stdin`` is read. If no output file is specified, ``stdout`` is written to. In ``dry_run``
    mode, output is written to ``stderr``.
    
    :param input_file: Path to atparse file (``None`` => read ``stdin``).
    :param output_file: Path to the file to write the converted template to.
    :param dry_run: Run in dry-run mode?
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True``.



The template tool works with atparse templates like the one shown below.

In [12]:
%%bash
cat 'fixtures/template/translate-template.yaml'

flowers:
  roses: @[color1]
  violets: @[color2]


We can translate this file to a Jinja2 template by passing appropriate `input_file` and `output_file` (either `str` or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a>) values to `template.render()`.

In [13]:
template.translate(
    input_file=Path('fixtures/template/translate-template.yaml'),
    output_file='fixtures/template/complete-translate.yaml'
)

True

Now we have created a Jinja2 template that can be rendered using `template.render()` or `template.render_to_str()`.

In [14]:
%%bash
cat 'fixtures/template/complete-translate.yaml'

flowers:
  roses: {{ color1 }}
  violets: {{ color2 }}
